# Análise Estatística – Cartões (Base: estoque_cartoes.db)

Este notebook tem como objetivo realizar uma análise estatística descritiva sobre os dados de cartões emitidos e ativos, segmentados por trimestre.

**Foco:**
- Médias, medianas, modas
- Desvios padrão
- Agrupamentos por mês e trimestre


## Carregamento dos Dados

Aqui conectamos ao banco de dados SQLite `estoque_cartoes.db` e carregamos os dados da tabela `estoque_cartoes` para um DataFrame.


In [14]:
import pandas as pd
import sqlite3

con = sqlite3.connect("datasets/estoque_cartoes.db")
df = pd.read_sql("SELECT * FROM estoque_cartoes", con)
con.close()

In [13]:
df.head()

,trimestre,nomebandeira,nomefuncao,produto,qtdcartoesemitidos,qtdcartoesativos,ano_mes,trimestre_periodo
0,2023-03-01,American Express,Crédito,Intermediário,433549,335542,2023-03,2023Q1
1,2023-03-01,American Express,Crédito,Empresarial,26598,18382,2023-03,2023Q1
2,2023-03-01,American Express,Crédito,Corporativo,68432,49995,2023-03,2023Q1
3,2023-09-01,American Express,Crédito,Empresarial,27741,19039,2023-09,2023Q3
4,2024-06-01,American Express,Crédito,Premium,537851,414197,2024-06,2024Q2


## Conversão da Coluna `trimestre`

A coluna `trimestre` está em formato texto como "20231", "20232", etc. Vamos converter para date de cada trimestre para facilitar o agrupamento por tempo.


In [ ]:
def converter_trimestre_para_data(trimestre_str):
    trimestre_str = str(trimestre_str)
    ano = int(trimestre_str[:4])
    trimestre = int(trimestre_str[-1])
    mes_final = trimestre * 3
    return pd.Timestamp(year=ano, month=mes_final, day=1)

df["trimestre"] = df["trimestre"].apply(converter_trimestre_para_data)

## Estatísticas Descritivas

Exibimos um resumo estatístico geral das colunas numéricas da base de dados.


In [ ]:
df.describe(include="all")

,trimestre,nomebandeira,nomefuncao,produto,qtdcartoesemitidos,qtdcartoesativos
count,100,100,100,100,1.000000e+02,1.000000e+02
unique,NaN,7,3,8,NaN,NaN
top,NaN,MasterCard,Crédito,Empresarial,NaN,NaN
freq,NaN,34,47,17,NaN,NaN
mean,2024-01-09 08:38:24,NaN,NaN,NaN,1.364954e+07,4.565303e+06
min,2023-03-01 00:00:00,NaN,NaN,NaN,2.180000e+02,0.000000e+00
25%,2023-06-01 00:00:00,NaN,NaN,NaN,4.223892e+05,1.425770e+05
50%,2024-03-01 00:00:00,NaN,NaN,NaN,2.017354e+06,6.389785e+05
75%,2024-09-01 00:00:00,NaN,NaN,NaN,9.618478e+06,3.449852e+06
max,2024-12-01 00:00:00,NaN,NaN,NaN,2.084751e+08,4.766928e+07


## Tendência Central

Aqui calculamos as principais medidas de tendência central:
- Média
- Mediana
- Moda

Os resultados são formatados com separadores de milhar.


In [ ]:
colunas = ["qtdcartoesemitidos", "qtdcartoesativos"]

media = df[colunas].mean().apply(lambda x: f"{x:,.2f}")
mediana = df[colunas].median().apply(lambda x: f"{x:,.2f}")
moda = df[colunas].mode().iloc[0].apply(lambda x: f"{x:,.2f}")

print("Médias:\n", media)
print("\nMedianas:\n", mediana)
print("\nModas:\n", moda)

Médias:
 qtdcartoesemitidos    13,649,544.93
qtdcartoesativos       4,565,302.72
dtype: object

Medianas:
 qtdcartoesemitidos    2,017,354.00
qtdcartoesativos        638,978.50
dtype: object

Modas:
 qtdcartoesemitidos    218.00
qtdcartoesativos        0.00
Name: 0, dtype: object


## Desvio Padrão

O desvio padrão mede a dispersão dos dados em torno da média.

In [ ]:
desvio = df[colunas].std().apply(lambda x: f"{x:,.2f}")
print("Desvio padrão:\n", desvio)

Desvio padrão:
 qtdcartoesemitidos    32,090,516.97
qtdcartoesativos       8,931,789.88
dtype: object


## Agrupamento por Mês

Agrupamos os dados por mês (`ano_mes`) e calculamos as médias mensais para os cartões emitidos e ativos.


In [ ]:
df['ano_mes'] = df['trimestre'].dt.to_period('M')
media_mensal = df.groupby('ano_mes')[colunas].mean().applymap(lambda x: f"{x:,.2f}")
media_mensal

C:\Users\DELL\AppData\Local\Temp\ipykernel_11768\3497684210.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  media_mensal = df.groupby('ano_mes')[colunas].mean().applymap(lambda x: f"{x:,.2f}")


,qtdcartoesemitidos,qtdcartoesativos
ano_mes,,
2023-03,"7,395,850.54","2,371,899.00"
2023-06,"14,438,696.40","5,944,957.33"
2023-09,"14,484,573.15","5,194,776.31"
2023-12,"37,431,728.83","8,564,547.50"
2024-03,"9,983,917.47","4,794,751.33"
2024-06,"10,592,426.08","2,681,535.33"
2024-09,"4,147,889.28","1,582,701.28"
2024-12,"35,976,035.50","10,626,691.38"


## Agrupamento por Trimestre

Agrupamos os dados por trimestre e calculamos as médias trimestrais para cartões emitidos e ativos.


In [ ]:
df['trimestre_periodo'] = df['trimestre'].dt.to_period('Q')
media_trimestral = df.groupby('trimestre_periodo')[colunas].mean().applymap(lambda x: f"{x:,.2f}")
media_trimestral

C:\Users\DELL\AppData\Local\Temp\ipykernel_11768\1326899333.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  media_trimestral = df.groupby('trimestre_periodo')[colunas].mean().applymap(lambda x: f"{x:,.2f}")


,qtdcartoesemitidos,qtdcartoesativos
trimestre_periodo,,
2023Q1,"7,395,850.54","2,371,899.00"
2023Q2,"14,438,696.40","5,944,957.33"
2023Q3,"14,484,573.15","5,194,776.31"
2023Q4,"37,431,728.83","8,564,547.50"
2024Q1,"9,983,917.47","4,794,751.33"
2024Q2,"10,592,426.08","2,681,535.33"
2024Q3,"4,147,889.28","1,582,701.28"
2024Q4,"35,976,035.50","10,626,691.38"
